# Dit RAG ? 🤖

## Récupération de la base de données vectorielle

On va déjà récupérer la base de données vectorielle des publis DIRAG

In [1]:
import os

from s3fs import S3FileSystem

fs = S3FileSystem(
    client_kwargs={"endpoint_url": f"https://{os.environ['AWS_S3_ENDPOINT']}"},
    key=os.environ["AWS_ACCESS_KEY_ID"],
    secret=os.environ["AWS_SECRET_ACCESS_KEY"],
)

In [2]:
s3_path = "projet-llm-insee-open-data/data/chroma_database/experiment/dirag/"
local_path = "data/chroma_db_checkpoint/"

fs.download(
    rpath=s3_path,
    lpath=local_path,
    recursive=True,
)

OSError: [Errno 5] An error occurred (InvalidTokenId) when calling the ListObjectsV2 operation: The security token included in the request is invalid

On _load_ la base dans `Python` grâce à `langchain`

In [3]:
from src.db_building.build_database import load_vector_database_from_local

db = load_vector_database_from_local(
    persist_directory="data/chroma_db_checkpoint/", embedding_model="OrdalieTech/Solon-embeddings-large-0.1"
)

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-13 12:50:24.614 | INFO     | src.db_building.build_database:load_vector_database_from_local:125 - Loading Chroma vector database from local session
2025-02-13 12:50:27.377 | INFO     | src.db_building.build_database:load_vector_database_from_local:145 - Database (collection insee_data) reloaded from directory data/chroma_db_checkpoint/


On vérifie juste que ça fonctionne sur une requête simple

In [4]:
db.similarity_search("quel est le secteur dominant en guadeloupe?")

Batches: 100%|██████████| 1/1 [00:02<00:00,  2.73s/it]


[Document(id='116f4dd4-9e68-4685-a466-59e1db88b52d', metadata={'categorie': 'Publications grand public', 'collection': 'Insee Analyses', 'dateDiffusion': '2017-03-21 16:00', 'libelleAffichageGeo': 'Guadeloupe', 'theme': 'Dynamique des territoires', 'titre': 'Nord Grande-Terre\xa0: une expansion urbaine, une identité rurale', 'url': 'https://www.insee.fr/fr/statistiques/2661121'}, page_content='. Cette prédominance sectorielle (figure\xa04) expose l’économie compte tenu de son poids relatif, de l’environnement concurrentiel du secteur et de l’aléa climatique. La résistance de l’agriculture sur le territoire est portée par les grandes cultures (notamment la canne). L’hébergement médico-social et l’action sociale représentent près de 9\xa0% du total des emplois (7\xa0% dans le reste de la Guadeloupe) en relation avec le vieillissement de la population et le faible niveau de revenus des ménages.\n\nPar ailleurs, le secteur tertiaire est fortement sous-représenté. Les cinq activités que son

On transforme en `retriever` langchain

In [7]:
from src.db_building import load_retriever

retriever, vectorstore = load_retriever(
    vectorstore=db,
    retriever_params={"search_type": "similarity", "search_kwargs": {"k": 10}},
)

In [ ]:
retrieved_docs = retriever.invoke("subventions aux entreprises")


Batches: 100%|██████████| 1/1 [00:00<00:00, 16.85it/s]


,categorie,collection,dateDiffusion,libelleAffichageGeo,theme,titre,url
0,Communiqués de presse,,2016-05-20 10:36,Guyane,,Les aides publiques aux entreprises soutiennen...,https://www.insee.fr/fr/information/2128527
1,Communiqués de presse,,2016-05-20 10:36,Guyane,,Les aides publiques aux entreprises soutiennen...,https://www.insee.fr/fr/information/2128527
2,Publications grand public,Insee Analyses,2015-06-09 15:00,Guyane,Caractéristiques des entreprises,"Les aides publiques à l’emploi, au financement...",https://www.insee.fr/fr/statistiques/1285882
3,Publications grand public,Insee Analyses,2015-06-09 15:00,Guyane,Caractéristiques des entreprises,"Les aides publiques à l’emploi, au financement...",https://www.insee.fr/fr/statistiques/1285882
4,Publications grand public,Insee Analyses,2014-11-18 15:59,Guyane,"Mondialisation, compétitivité et innovation","En Guyane, six entreprises sur dix ont innové ...",https://www.insee.fr/fr/statistiques/1285698
5,Publications grand public,Insee Analyses,2014-11-18 15:59,Guyane,"Mondialisation, compétitivité et innovation","En Guyane, six entreprises sur dix ont innové ...",https://www.insee.fr/fr/statistiques/1285698
6,Publications grand public,Insee Analyses,2016-05-20 16:00,Guyane,Démographie et créations des entreprises,Les aides publiques aux entreprises soutiennen...,https://www.insee.fr/fr/statistiques/2019605
7,Publications grand public,Insee Analyses,2016-05-20 16:00,Guyane,Démographie et créations des entreprises,Les aides publiques aux entreprises soutiennen...,https://www.insee.fr/fr/statistiques/2019605
8,Publications grand public,Insee Analyses,2015-06-09 15:00,Guyane,Caractéristiques des entreprises,"Les aides publiques à l’emploi, au financement...",https://www.insee.fr/fr/statistiques/1285882
9,Publications grand public,Insee Analyses,2015-06-09 15:00,Guyane,Caractéristiques des entreprises,"Les aides publiques à l’emploi, au financement...",https://www.insee.fr/fr/statistiques/1285882


## Chain RAG vllm

In [ ]:
import requests

available_models = (
    requests.get("https://projet-llm-insee-open-data-vllm.user.lab.sspcloud.fr/v1/models").json().get("data")
)
available_models = [model["id"] for model in available_models]

available_models

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from src.utils import create_prompt_from_instructions, format_docs

system_instructions = """
Tu es un assistant spécialisé dans la statistique publique.
Tu réponds à des questions concernant les données de l'Insee, l'institut national statistique Français.

Réponds en FRANCAIS UNIQUEMENT. Utilise une mise en forme au format markdown.

En utilisant UNIQUEMENT les informations présentes dans le contexte, réponds de manière argumentée à la question posée.

La réponse doit être développée et citer l'ensemble de ses sources (titre et url de la publication) qui sont référencées à la fin.
Cite notamment l'url d'origine de la publication, dans un format markdown. un lien hypertext vers l'url de la source serait apprécié.

Cite 10 sources maximum.

Tu n'es pas obligé d'utiliser les sources les moins pertinentes.

Si tu ne peux pas induire ta réponse du contexte, ne réponds pas.

Voici le contexte sur lequel tu dois baser ta réponse :
Contexte: {context}
"""

question_instructions = """
Voici la question à laquelle tu dois répondre :
Question: {question}

Réponse:
"""

In [ ]:
from openai import OpenAI

# Set OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "https://projet-llm-insee-open-data-vllm.user.lab.sspcloud.fr/v1/"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)


def statbot_from_vllm_api(
    question, retriever, client, system_instructions=system_instructions, question_instructions=question_instructions
):
    prompt = create_prompt_from_instructions(system_instructions, question_instructions)
    context = format_docs(retriever.invoke(question))

    question_with_context = prompt.format(question=question, context=context)

    chat_response = client.chat.completions.create(
        model=available_models[0],
        messages=[
            {"role": "system", "content": system_instructions},
            {"role": "user", "content": question_with_context},
        ],
    )
    return chat_response, chat_response.choices[0].message.content

In [ ]:
from IPython.display import Markdown, display

_, answer = statbot_from_vllm_api(
    "Quelles sont les subventions spécifiques aux DOMS et leur effet sur le tissu productif ?", retriever, client
)
display(Markdown(answer.replace("   ", "")))

In [ ]:
_, answer = statbot_from_vllm_api("La Guadeloupe est-elle défavorisée par rapport à la Métropole ?", retriever, client)
display(Markdown(answer.replace("   ", "")))

In [ ]:
_, answer = statbot_from_vllm_api("y a quoi à manger à la cantine ce soir?", retriever, client)
display(Markdown(answer.replace("   ", "")))

## Chain RAG locale

In [ ]:
from langchain_community.llms import VLLM

from src.config import MODEL_TO_ARGS

LLM_MODEL = "mistralai/Mistral-7B-Instruct-v0.3"

llm = VLLM(model=LLM_MODEL, **MODEL_TO_ARGS.get(LLM_MODEL, {}))

In [ ]:
system_instructions = """
Tu es un assistant spécialisé dans la statistique publique.
Tu réponds à des questions concernant les données de l'Insee, l'institut national statistique Français.

Réponds en FRANCAIS UNIQUEMENT. Utilise une mise en forme au format markdown.

En utilisant UNIQUEMENT les informations présentes dans le contexte, réponds de manière argumentée à la question posée.

La réponse doit être développée et citer ses sources (titre et url de la publication) qui sont référencées à la fin.
Cite notamment l'url d'origine de la publication, dans un format markdown.

Cite 5 sources maximum.

Tu n'es pas obligé d'utiliser les sources les moins pertinentes.

Si tu ne peux pas induire ta réponse du contexte, ne réponds pas.

Voici le contexte sur lequel tu dois baser ta réponse :
Contexte: {context}
"""

question_instructions = """
Voici la question à laquelle tu dois répondre :
Question: {question}

Réponse:
"""

prompt = create_prompt_from_instructions(system_instructions, question_instructions)

rag_chain = {"context": retriever | format_docs, "question": RunnablePassthrough()} | prompt | llm | StrOutputParser()

In [ ]:
from IPython.display import Markdown, display

answer = rag_chain.invoke("Quel est le niveau de vie en Guadeloupe?")

display(Markdown(answer.replace("   ", "")))